In [1]:
from selenium import webdriver
import time

import config

In [2]:
def get_task_urls(driver, contest_url):
    
    driver.get(contest_url)
    rows = driver.find_elements_by_xpath('//table/tbody//tr')
    task_urls = []
    for row in rows:
        task_url = row.find_element_by_xpath('td/a').get_attribute('href')
        task_urls.append(task_url)
    return task_urls

In [3]:
def create_ipynb(driver, task_url, path_ipynb):
    
    driver.get(task_url)
    html = driver.page_source
    html = html.split('<div class="col-sm-12">\n\t\t')[-1]
    html = html.split('\n<span class="lang-en"')[0]
    html = html.split('<div class="a2a_kit a2a_kit_size_20 a2a_default_style pull-right"')[0]

    html = html.replace('class="mjx-chtml MathJax_CHTML MJXc-processed" tabindex="0"',
                        'class="mjx-chtml MathJax_CHTML MJXc-processed" tabindex="0" style="display: none;"')
    html = html.replace('<span class="btn-copy btn-pre" tabindex="0" data-toggle="tooltip" data-trigger="manual" title=""',
                        '<span class="btn-copy btn-pre" tabindex="0" data-toggle="tooltip" data-trigger="manual" title="" style="display: none;"')
    html = html.replace('<pre class="prettyprint linenums source-code prettyprinted" style="">',
                        '<pre class="prettyprint linenums source-code prettyprinted" style="display: none;">')
    html = html.replace('"', "'")
    html = html.replace('\t', '')
    html = html.replace('\\', '\\\\')
    html = html.replace('\n', ' \\n",\n"')
    html = '"' + html + '\\n"'

    head = '{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "metadata": {},\n   "source": [\n'
    tail = '   ]\n  }\n ],\n "metadata": {\n  "kernelspec": {\n   "display_name": "Python 3",\n   "language": "python",\n   "name": "python3"\n  },\n  "language_info": {\n   "codemirror_mode": {\n    "name": "ipython",\n    "version": 3\n   },\n   "file_extension": ".py",\n   "mimetype": "text/x-python",\n   "name": "python",\n   "nbconvert_exporter": "python",\n   "pygments_lexer": "ipython3",\n   "version": "3.7.1"\n  }\n },\n "nbformat": 4,\n "nbformat_minor": 2\n}\n'

    with open(path_ipynb, 'w', newline='\n', encoding='utf-8') as f:
        f.write(head + html + tail)

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(config.path_to_chromedriver, chrome_options=options)
driver.set_window_size(1920, 1080)

alphabet = 'abcdefghijklmnopqrstuvwxyz'
for i in range(1, 139):
    contest_url = f'https://atcoder.jp/contests/abc{i:03}/tasks'
    time.sleep(1)
    task_urls = get_task_urls(driver, contest_url)
    for j, task_url in enumerate(task_urls):
        time.sleep(1)
        path_ipynb = f'contests/ABC/ABC{i:03}/ABC{i:03}_{alphabet[j]}.ipynb'
        create_ipynb(driver, task_url, path_ipynb)
driver.close()

C:\Users\taiki\Anaconda3\Lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.
